<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/add_sptoken_for_mlm12042020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 890kB 56.4MB/s 
     |████████████████████████████████| 2.9MB 57.2MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [4]:
df['comment']='<'+df['subtask_a']+'> '+df['tweet']

In [5]:
df.head()

,tweet,subtask_a,subtask_b,subtask_c,comment
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,<OFF> @USER She should ask a few native Americ...
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,<OFF> @USER @USER Go home you’re drunk!!! @USE...
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,<NOT> Amazon is investigating Chinese employee...
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"<OFF> @USER Someone should'veTaken"" this piece..."
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,<NOT> @USER @USER Obama wanted liberals &amp; ...


In [6]:

M = df['comment']
 
M.to_csv('data.txt',header=False,index=False)

In [7]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens':["<OFF>","<NOT>"]})

2

In [8]:
from transformers import BertForMaskedLM


model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(28998, 768)

In [10]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="data.txt",
    block_size=128,
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [14]:
from transformers import Trainer, TrainingArguments

model.cuda()
training_args = TrainingArguments(
    output_dir="./mlm",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=16,
    save_steps=1_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    # prediction_loss_only=True,
)

In [15]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,1.688927
1000,1.627137
1500,1.657672


CPU times: user 6min 4s, sys: 3min 34s, total: 9min 38s
Wall time: 9min 43s


TrainOutput(global_step=1656, training_loss=1.6684534192661156)

In [17]:
trainer.save_model("./mlm")

In [18]:

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./mlm",
    tokenizer=tokenizer
)

Some weights of BertModel were not initialized from the model checkpoint at ./mlm and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
fill_mask("<OFF> @USER She should [MASK] a few native Americans what their take on this is.")

[{'score': 0.6004376411437988,
  'sequence': '[CLS] <OFF> @ USER She should be a few native Americans what their take on this is. [SEP]',
  'token': 1129,
  'token_str': 'be'},
 {'score': 0.17993400990962982,
  'sequence': '[CLS] <OFF> @ USER She should have a few native Americans what their take on this is. [SEP]',
  'token': 1138,
  'token_str': 'have'},
 {'score': 0.08353453129529953,
  'sequence': '[CLS] <OFF> @ USER She should not a few native Americans what their take on this is. [SEP]',
  'token': 1136,
  'token_str': 'not'},
 {'score': 0.009834140539169312,
  'sequence': '[CLS] <OFF> @ USER She should make a few native Americans what their take on this is. [SEP]',
  'token': 1294,
  'token_str': 'make'},
 {'score': 0.004605247173458338,
  'sequence': '[CLS] <OFF> @ USER She should just a few native Americans what their take on this is. [SEP]',
  'token': 1198,
  'token_str': 'just'}]